# Single Substitution Simulation Run Trace

(c) 2020 Tom Röschinger. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

***

In [6]:
using LinearAlgebra, Jevo, Jedi, Plots, LambertW, Statistics
Jedi.default_gr!()

# Parameters
steps = 1* 10^8
rho = 3
l_0 = 15
N = 100
nu = 0
ϵ = 2
n = 4
emat = ϵ * (ones(4, 4) - Matrix{Float64}(I, 4, 4))
f0 = 100/2N

0.5

In [8]:
# Length scale
l0(ϵ, n, f0=f0) = 1/ϵ * lambertw(2 * ϵ^2 * N * 10 * f0 * (n-1)/n^2* exp(5 * ϵ))
l0(κ, l, ϵ, n, f0=f0) = 1/ϵ * lambertw(2 * ϵ^2 * N * l * f0 * (n-1)/n^2 * exp(5 * ϵ)/(1 + κ))

# Find length cost to set length in equilibrium (derived from genetic load)
#lambda(l_opt) = l0()^2/l_opt^2 * n^2 / (n-1) * 1/ϵ

# Find length cost to set length in equilibrium (derived from genetic load modified by length dynamics)

lambda_new(l_opt, ϵ, n, f0=f0) = l0(ϵ, n, f0)^3 / l_opt^2 * n^2 / (n-1) * 1/2
lambda_new(l_opt,κ, l, ϵ, n, f0=f0) = l0(κ, l, ϵ, n, f0)^3 / l_opt^2 * n^2 / (n-1) * 1/2

#l_opt_static(κ, ϵ, n) = l0(ϵ, n) * sqrt(n^2 / ((n - 1) * ϵ * lambda(10)) * (1+κ))
l_opt_dynamic(κ, ϵ, n, f0=f0) = l0(ϵ, n, f0)^(3/2) * sqrt(n^2 / ((n - 1) * 2 * lambda_new(10, ϵ, n, f0)) * (1+κ))

F = Jevo.fermi_fitness(f0=f0, fl=lambda_new(10, ϵ, n)/(2N*l0(ϵ, n, f0)), epsilon=ϵ);

In [9]:

pop = Jevo.mono_pop(N=100, l=l_0)
Jevo.initiate!(pop, emat)
t0 = zeros(2, steps)
for i in 1:steps
    Jevo.bp_substitution!(pop, emat, F)
    if rand() < 0/N
        Jevo.driver_mutation!(pop)
    end
    if rand() < 1/N^2
        Jevo.l_substitution!(pop, emat, F)
    end
    # Recover lost sites
    if length(pop.seqs) < 7
        Jevo.initiate!(pop, emat)
    end
    t0[:, i] = [Jevo.get_energy(pop, emat), length(pop.seqs)] 
end


pop = Jevo.mono_pop(N=100, l=l_0)
Jevo.initiate!(pop, emat)
t5 = zeros(2, steps)
for i in 1:steps
    Jevo.bp_substitution!(pop, emat, F)
    if rand() < rho/N
        Jevo.driver_mutation!(pop)
    end
    if rand() < rho/N^2
        Jevo.l_substitution!(pop, emat, F)
    end
    # Recover lost sites
    if length(pop.seqs) < 7
        Jevo.initiate!(pop, emat)
    end
    t5[:, i] = [Jevo.get_energy(pop, emat), length(pop.seqs)] 
end


In [14]:
plot(t0[2, 1:1000:end], label=0)
#plot!(t1[2, 1:150:end], label=1)
plot!(t5[2, 1:1000:end], label=rho)

In [12]:
mean(t5[2, 1:1000:end])

14.52395